# RBA Selected Tables

Worth reading

*  [Updates to Australia's Financial Aggregates](https://www.rba.gov.au/publications/bulletin/2019/mar/updates-to-australias-financial-aggregates.html)

## Set-Up

In [1]:
# standard library imports
from pathlib import Path
from functools import cache
import textwrap
import math

# usual data science imports
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns  # type: ignore
import readabs as ra
from babel.numbers import get_currency_name

In [2]:
# local imports
from plotting import (
    clear_chart_dir,
    finalise_plot,
    line_plot,
    plot_covid_recovery,
    plot_series_highlighted,
    set_chart_dir,
)

In [3]:
# display charts in notebook
SHOW = False
FILE_TYPE = "png"
pd.options.display.max_rows = 100

### Some useful constants

In [4]:
TODAY = pd.Timestamp("today")
TOMORROW = TODAY + pd.DateOffset(days=1)

# multi-time-period charts
YEARS5_PERIOD_Y = 5  # years
YEARS5_PERIOD_M = 3  # months
YEARS5 = TODAY - pd.DateOffset(years=YEARS5_PERIOD_Y, months=YEARS5_PERIOD_M)

CURRENT_CYCLE = pd.Timestamp("2022-04-01")

V_RECENCY_PERIOD = 370  # days
YEARS1 = TODAY - pd.DateOffset(days=V_RECENCY_PERIOD)

VV_RECENCY_PERIOD = 180  # days
MONTHS6 = TODAY - pd.DateOffset(days=VV_RECENCY_PERIOD)

TEN_YEARS = TODAY - pd.DateOffset(years=10)

# Where to put the charts
CHART_DIR = "./CHARTS/RBA/"
Path(CHART_DIR).mkdir(parents=True, exist_ok=True)
clear_chart_dir(CHART_DIR)
set_chart_dir(CHART_DIR)

# plotting stuff
plt.style.use("fivethirtyeight")
SOURCE = "Source: RBA"

### List available data tables - but only if needed

In [5]:
NEED_HELP = False
if NEED_HELP:
    ra.print_rba_catalogue()

### Utility functions

In [6]:
def print_last_date(df: pd.DataFrame) -> None:
    """Print the last date index in a DataFrame."""

    print(f"Last date: {df.index[-1]}")

## Monetary Policy - Official Cash Rate (A2)

In [7]:
def plot_cash_rate():
    """Produce official cash rate (OCR) chart."""

    # get the data
    ocr_rba = ra.read_rba_ocr()

    # plot full series and recent
    line_plot(
        ocr_rba,
        starts=["1993-01-01", YEARS5, CURRENT_CYCLE],
        title="RBA Official Cash Rate",
        drawstyle="steps-post",
        ylabel="Per cent",
        zero_y=True,
        width=2,
        rfooter=f"{SOURCE} A2",
        lfooter="Australia. Monthly data. ",
        show=SHOW,
        file_type=FILE_TYPE,
        pre_tag="a2-",
    )

    # plot tightening cycles
    from_date = "1993-01-01"  # Start of RBA inflation-targeting approach
    series_93 = ocr_rba[ocr_rba.index >= from_date]
    ax = plot_series_highlighted(series_93)
    finalise_plot(
        ax,
        title="RBA Official Cash Rate - Tightening Cycles",
        ylabel="Per cent",
        rfooter=f"{SOURCE} A2",
        lfooter=f"Australia. Monthly data. Right end-point: {series_93.iloc[-1]} per cent. ",
        show=SHOW,
        file_type=FILE_TYPE,
        pre_tag="a2-",
    )


plot_cash_rate()

## OCR Futures - zero-coupon forward rates (F17)
Note: series updated monthly.

In [8]:
@cache
def get_zero_coupon_data(forward_years=1.5) -> pd.DataFrame:
    """Get zero coupon interest rates data."""

    # Note: name is misleading - data is from 2017
    source = "F17"

    # get data
    zero_coupon_tuple = ra.read_rba_table(source)
    zero_coupon, zc_meta = zero_coupon_tuple

    # extract the periods of interest
    periods = (
        zc_meta.Title.str.split(" – ")
        .str[-1]
        .str.replace(" yrs", "")
        .str.replace(" yr", "")
        .astype(float)
    )
    wanted = periods[periods <= forward_years].index
    periods = periods.loc[wanted]
    # titles = zc_meta.loc[wanted].Title
    data = zero_coupon.loc[:, wanted]
    data.columns = periods  # data in tight format

    # convert to a sparse matrix for plotting
    matrix = {}
    for date, series in data.iterrows():
        period = pd.Period(date, freq="D")
        series.index = (
            pd.to_timedelta((series.index * 365).astype(int), unit="D") + period
        )
        matrix[period] = series
    frame = pd.DataFrame(matrix)

    return frame

In [9]:
def plot_zc_minmax() -> None:
    """Plot zero coupon interest rates min and max."""

    # get the data
    zc_data = get_zero_coupon_data()

    # plot min and max
    ax = zc_data.max().plot(label="Max", color="darkorange", lw=1.5)
    zc_data.min().plot(ax=ax, label="Min", color="cornflowerblue", lw=1.5)
    finalise_plot(
        ax,
        title="Zero-coupon Forward Rates - Min/Max over forward 18m",
        ylabel="Rate (%/year)",
        rfooter=f"{SOURCE} F17",
        lfooter=f"Australia. Daily data. Data to {zc_data.columns[-1]}. ",
        legend={"loc": "upper left", "fontsize": "x-small"},
        show=SHOW,
        file_type=FILE_TYPE,
        pre_tag="f17-",
    )


plot_zc_minmax()

In [10]:
def plot_zc_monthly():
    """Plot zero coupon interest rates."""

    df = get_zero_coupon_data().T.to_timestamp()
    grouped = (
        df.groupby(by=[df.index.year, df.index.month])
        .last(skipna=False)  # last day of month
        .dropna(how="all", axis=1)  # drop columns with all NaNs
        .T
    )
    num_columns = len(grouped.columns)
    colors = sns.color_palette("cool", num_columns)
    delegend = [f"_{x}" for x in grouped.columns]
    grouped.columns = delegend

    ax = None
    for i, col in enumerate(grouped.columns):
        ax = grouped[col].dropna().plot(ax=ax, lw=1, c=colors[i])
    ocr = ra.read_rba_ocr().asfreq("D")
    ocr = ocr[ocr.index >= grouped.index.min()]
    ocr.plot(
        ax=ax,
        lw=2.5,
        c="r",
        label="RBA Official Cash Rate",
        drawstyle="steps-post",
    )
    finalise_plot(
        ax,
        title="EOM Zero-coupon Forward Rates (over forward 18 months)",
        ylabel="Rate (%/year)",
        rfooter=f"{SOURCE} A2 F17",
        lfooter=f"Australia. EOM=End of month. Data to {df.index[-1].date()}. ",
        legend={"loc": "upper left", "fontsize": "x-small"},
        show=SHOW,
        file_type=FILE_TYPE,
    )


plot_zc_monthly()

In [11]:
def plot_zc_daily():
    """Plot zero coupon interest rates on a daily basis."""

    sparse = get_zero_coupon_data()
    num_columns = len(sparse.columns)
    colors = sns.color_palette("cool", num_columns)
    ax = None
    for i, col in enumerate(sparse.columns):
        ax = sparse[col].dropna().plot(ax=ax, lw=0.2, alpha=0.5, c=colors[i])
    finalise_plot(
        ax,
        title="Zero-coupon Forward Rates (over forward 18 months)",
        ylabel="Rate (%/year)",
        rfooter=f"{SOURCE} F17",
        lfooter=f"Australia. Daily data. Data to {sparse.columns[-1]}. ",
        show=SHOW,
        file_type=FILE_TYPE,
        pre_tag="f17-",
    )


plot_zc_daily()

## Housing Loan Payments (E13)

In [12]:
def housing_repayments():
    """Produce housing loan repayment charts."""

    e13_data, e13_meta = ra.read_rba_table("E13")
    print_last_date(e13_data)

    # Note - some data is labeled monthly and some data is labeled quarterly
    # freqs = {"Quarterly": "Q", "Monthly": "M"}
    for _, row in e13_meta.iterrows():
        title = row["Title"]
        unit = row["Units"]
        series_id = row["Series ID"]
        series_type = row["Type"]
        # freq = freqs[row["Frequency"]]
        series = e13_data[series_id].astype(float).dropna()
        series, unit = ra.recalibrate(series, unit)

        split_title_len = 50  # characters
        if len(title) > split_title_len:
            title = "\n".join(title.rsplit(";", 1))

        line_plot(
            series,
            starts=(None, YEARS5),
            tags=("", "-recent"),
            pre_tag="e13-",
            title=title,
            ylabel=unit,
            rfooter=f"{SOURCE} E13",
            lfooter=f"Australia. {series_type}. Endpoint at {series.index[-1]}: "
            f"{series.iloc[-1]:.02f} {unit}. ",
            width=2,
            show=SHOW,
            file_type=FILE_TYPE,
        )


housing_repayments()

Last date: 2024Q3
recalibrate(): Units not appropriately calibrated: Per cent
recalibrate(): Units not appropriately calibrated: Per cent
recalibrate(): Units not appropriately calibrated: Per cent
recalibrate(): Units not appropriately calibrated: Per cent
recalibrate(): Units not appropriately calibrated: Per cent


## Key market interest rates (F1)

In [13]:
def market_rates():
    """Produce market interest rates charts."""

    # get the dataset
    f1_data, f1_meta = ra.read_rba_table("F1")
    print_last_date(f1_data)

    # get the specific data we want, for the very recent past
    wanted = [
        "Cash Rate Target",
        #'Interbank Overnight Cash Rate',
        "EOD 1-month BABs/NCDs",
        # '1-month OIS',  # discontinued December 2022
        # '1-month Treasury Note',
    ]
    series_ids = [f1_meta[f1_meta.Title == x].index[0] for x in wanted]
    plotable_data = (
        f1_data[series_ids]
        .rename(dict(zip(series_ids, wanted)), axis=1)
        .dropna(how="all", axis=1)
        .dropna(how="all", axis=0)
        # .ffill()
        .infer_objects(copy=False)
    )

    # And plot
    title = "Australia - short-term Interest rates"
    line_plot(
        plotable_data,
        starts=CURRENT_CYCLE,
        title=title,
        drawstyle="steps-post",
        ylabel="Per cent",
        rfooter=f"{SOURCE} F1 Daily",
        lfooter="Key: EOD = end of day; "
        "BABs/NCDs = Bank Accepted Bills / Negotiable Certificates of Deposit.",
        width=2,
        show=SHOW,
        file_type=FILE_TYPE,
        pre_tag="f1-",
    )


market_rates()

Last date: 2025-01-14


## Australian Government Bonds (F2)

Data updated on Fridays (to two business days earlier.)

### Utility functions

In [14]:
def get_bonds(meta, data):
    """Get Bonds data."""

    bond_yield = meta[
        meta.Title.str.contains("Australian|Commonwealth")
        & meta.Title.str.contains("year")
    ]
    bond_yield_labels = bond_yield.Title.str.replace(
        "Australian Government ", ""
    ).str.replace("Commonwealth Government ", "")
    bond_yield_data = data[bond_yield_labels.index]
    bond_yield_data.columns = bond_yield_labels

    return bond_yield_data

In [15]:
def plot_yields(data, period, table, show=SHOW):
    """Plot full series and recent."""

    title = "Capital Market Yields - Australian Government Bonds"
    line_plot(
        data,
        starts=[None, MONTHS6],
        tags=f"{table}-{period}",
        width=1.5,
        drawstyle="steps-post",
        title=title,
        ylabel="Per cent per annum",
        rfooter=f"{SOURCE} {table} {period}",
        lfooter=f"Australian Government Bonds. Data up to {data.index[-1]}. ",
        show=show,
        file_type=FILE_TYPE,
        pre_tag="f2-",
    )

In [16]:
def inversion_plot(data, lo, hi, bonds_daily, show=SHOW):
    """Plot bond rate inversions, ehere they occur."""

    t_lo = f"{lo} year bond"
    t_hi = f"{hi} year bond"

    inversions = bonds_daily[t_hi] - bonds_daily[t_lo]
    inversions = -inversions.where(inversions < 0, other=0)
    line_plot(
        inversions,
        starts=[None, MONTHS6],
        title=f"Capital Market Yield Inversions [({hi}-year - {lo}-year) * -1]",
        ylabel="% points difference",
        rfooter=f"{SOURCE} F2 Daily",
        lfooter=f"Australian Government Bonds. Data up to {data.index[-1]}. ",
        show=show,
        file_type=FILE_TYPE,
        pre_tag="f2-",
    )

In [17]:
def bonds():
    """Produce bond market plots."""

    f2_data, f2_meta = ra.read_rba_table("F2")
    print_last_date(f2_data)
    bonds_daily = get_bonds(f2_meta, f2_data)
    plot_yields(bonds_daily, "Daily", "F2")
    inversion_plot(f2_data, 2, 3, bonds_daily)
    inversion_plot(f2_data, 2, 5, bonds_daily)
    inversion_plot(f2_data, 2, 10, bonds_daily)


bonds()

Last date: 2025-01-08


In [18]:
def plot_long_run_bonds() -> None:
    """Plot long run bond yields - 10 year Bonds."""

    tables = (
        "Z:F2-Daily-2013",
        "F2",
    )
    combined = pd.DataFrame()
    for t in tables:
        data, _meta = ra.read_rba_table(t)
        if combined.empty:
            combined = data
        else:
            combined = pd.concat([combined, data], axis=0)
    combined = combined.sort_index()
    ax = combined["FCMYGBAG10D"].plot(lw=1)
    ax.text(
        combined.index[-1],
        combined["FCMYGBAG10D"].iloc[-1],
        f" {combined['FCMYGBAG10D'].iloc[-1]:.02f}",
        va="center",
        ha="left",
        fontsize="x-small",
    )
    finalise_plot(
        ax,
        title="10 Year Australian Government Bond Yields",
        ylabel="Per cent per annum",
        rfooter=f"{SOURCE} F2 Daily",
        lfooter=f"Australian Government Bonds. Data up to {combined.index[-1]}. ",
        show=SHOW,
        file_type=FILE_TYPE,
    )


plot_long_run_bonds()

## Growth in credit aggregates (D1)

In [19]:
def credit_aggregates():
    """Produce credit aggregate charts."""

    d1_data, d1_meta = ra.read_rba_table("D1")
    print_last_date(d1_data)

    chart_set = [
        "Credit; Housing; Monthly growth",
        "Credit; Owner-occupier housing; Monthly growth",
        "Credit; Investor housing; Monthly growth",
        "Credit; Other personal; Monthly growth",
        # "Credit; Business; Monthly growth",
        # "Credit; Total; Monthly growth",
    ]

    for label in chart_set:
        name = d1_meta[d1_meta.Title == label].index[0]
        series_type = d1_meta[d1_meta.Title == label].Type.iloc[0]
        series = d1_data[name]

        line_plot(
            series,
            starts=[None, YEARS5],
            y0=True,
            title=label,
            ylabel="Per cent",
            rfooter=f"{SOURCE} D1",
            lfooter=series_type,
            show=SHOW,
            file_type=FILE_TYPE,
            pre_tag="d1-",
        )


credit_aggregates()

Last date: 2024-11


## Money supply (D3)

In [20]:
def money_supply():
    """Produce money supply charts."""

    # data capture
    d3_data, d3_meta = ra.read_rba_table("D3")
    print_last_date(d3_data)

    # plot Money Supply
    to_get = ["Currency: Seasonally adjusted", "Money base", "M1", "M3", "Broad money"]
    gotten = {t: d3_data[d3_meta[d3_meta.Title == t].index[0]] for t in to_get}
    m_units = {t: d3_meta[d3_meta["Title"] == t].Units.iloc[0] for t in to_get}

    for title, series in gotten.items():
        series = series.dropna().astype(float)
        series, m_unit = ra.recalibrate(series, m_units[title])
        series.name = title
        chart_title = f"{title} - Money Supply"

        aux_kwargs = {}
        aux_kwargs["start_r"], aux_kwargs["end_r"] = (
            ("2016-05-01", "2019-05-01")
            if title == "M1"
            else ("2017-01-01", "2020-01-01")
        )

        plot_covid_recovery(
            series,
            **aux_kwargs,
            title=chart_title,
            tags="COVID",
            ylabel=m_unit,
            rfooter=f"{SOURCE} D3",
            show=SHOW,
            file_type=FILE_TYPE,
            pre_tag="d3-",
        )

        line_plot(
            series,
            starts=[None, TEN_YEARS],
            title=chart_title,
            ylabel=m_unit,
            rfooter=f"{SOURCE} D3",
            show=SHOW,
            file_type=FILE_TYPE,
            pre_tag="d3-",
        )

        line_plot(
            series,
            starts=[None, TEN_YEARS],
            title=chart_title,
            ylabel=f"{m_unit} (log scale)",
            rfooter=f"{SOURCE} D3",
            yscale="log",
            show=SHOW,
            file_type=FILE_TYPE,
            pre_tag="d3-",
        )


money_supply()

Last date: 2024-11


## Hosuing lending rates (F6)

In [21]:
def housing_lending() -> None:
    """Plot housing lending rates."""

    wanted = (
        "FLRHOFTA",
        "FLRHOFVA",
        "FLRHOFFA",
    )
    table = "F6"
    data, meta = ra.read_rba_table(table)
    if not meta["Series ID"].is_unique:
        raise ValueError("Series IDs not unique")
    for w in wanted:
        series = data[meta[meta["Series ID"] == w].index[0]]
        series = series.dropna().astype(float)
        series, unit = ra.recalibrate(series, str(meta.loc[w, "Units"]))
        series.name = w
        line_plot(
            series,
            starts=[
                None,
            ],
            title=textwrap.fill(str(meta.loc[w, "Title"]), 60),
            ylabel=unit,
            rfooter=f"{SOURCE} {table}",
            lfooter="Australia. ",
            width=2,
            show=SHOW,
            file_type=FILE_TYPE,
            pre_tag="f6-",
        )


housing_lending()

recalibrate(): Units not appropriately calibrated: Per cent per annum
recalibrate(): Units not appropriately calibrated: Per cent per annum
recalibrate(): Units not appropriately calibrated: Per cent per annum


## Exchange rates (F11.1)

In [22]:
def find_local_min_max(series: pd.Series, window=132) -> pd.DataFrame:
    """Find local minima and maxima in a series, where
    the size of local is defined by the window parameter.
    Return a DataFrame of local minima and maxima,
    with columns "idx", "val" and "kind" (min, max or end)."""

    # validate input
    if not isinstance(series, pd.Series):
        raise TypeError("series must be a pandas Series")
    if not isinstance(series.index, pd.PeriodIndex):
        raise TypeError("series.index must be a PeriodIndex")
    if not series.index.is_unique:
        raise ValueError("series.index must be unique")
    if not series.index.is_monotonic_increasing:
        raise ValueError("series.index must be increasing")

    max_idx, min_idx = set(), set()
    for i in range(0, len(series), int(window / 2.5)):
        selection = series.iloc[i : (i + window)]
        if len(selection) < int(window / 2):
            break
        max_idx.add(selection.idxmax())
        min_idx.add(selection.idxmin())
    maximums = pd.PeriodIndex(sorted(list(max_idx)))
    minimums = pd.PeriodIndex(sorted(list(min_idx)))
    peak_set = pd.DataFrame(
        {
            "idx": maximums,
            "val": series[maximums],
            "kind": ["max"] * len(maximums),
        },
        index=maximums,
    )
    trough_set = pd.DataFrame(
        {
            "idx": minimums,
            "val": series[minimums],
            "kind": ["min"] * len(minimums),
        },
        index=minimums,
    )
    peaks_troughs = pd.concat([peak_set, trough_set]).sort_index(ascending=True)
    peaks_troughs = peaks_troughs[
        ~peaks_troughs.index.duplicated(keep="last")
    ]  # de-duplicate

    # remove excess peaks/troughs, when consequtive of the same kind
    peaks_troughs["group"] = (
        peaks_troughs["kind"] != peaks_troughs["kind"].shift(1)
    ).cumsum()
    reduced = (
        peaks_troughs.groupby(["group", "kind"])["val"]
        .agg(["min", "max", "idxmin", "idxmax"])
        .reset_index()
    )
    annotations = pd.concat(
        [
            reduced[reduced["kind"] == "min"][["idxmin", "min", "kind"]].rename(
                columns={"idxmin": "idx", "min": "val"},
            ),
            reduced[reduced["kind"] == "max"][["idxmax", "max", "kind"]].rename(
                columns={"idxmax": "idx", "max": "val"}
            ),
        ]
    ).sort_values(by="idx")
    annotations.index = annotations["idx"]  # type: ignore[assignment]

    # add endpoint
    if series.index[-1] == annotations.index[-1]:
        annotations.loc[series.index[-1], "kind"] = "end"  # type: ignore[index]
    else:
        annotations.loc[series.index[-1]] = (  # type: ignore[call-overload]
            [series.index[-1], series[series.index[-1]], "end"]  # type: ignore[call-overload]
        )

    return annotations

In [23]:
def annotate_chart_min_max(ax: plt.Axes, annotations: pd.DataFrame) -> None:
    """Annotate a chart using a table of annotations.
    which has the following columns: "idx", "val", "kind"."""

    # sanity check
    for col in ("idx", "val", "kind"):
        if col not in annotations.columns:
            raise ValueError(f"Column {col} not in annotations")

    # annotate the chart
    rounding = 3 - math.floor(math.log10(annotations["val"].min()))
    rounding = max(rounding, 0)
    for _index, row in annotations.iterrows():
        idx, val, kind = row.loc["idx"], row.loc["val"], row.loc["kind"]
        va, ha = "center", "left"  # end position
        if kind in ("min", "max"):
            va, ha = "top" if kind == "min" else "bottom", "center"
        ax.text(
            idx,
            val,
            f"{val:0.{rounding}f}",
            va=va,
            ha=ha,
            fontsize="x-small",
        )

In [24]:
def get_currency(title: str) -> str:
    """Get the currency name from the currency code within a title."""

    text = ""
    if "AUD" in title:
        code = title[4:7]
        text = get_currency_name(code, locale="en_AU")
    return text


In [25]:
def plot_exchange_rates():
    """Plot exchange rates over the short-term."""

    table = "F11.1"
    er_daily, er_daily_meta = ra.read_rba_table(table)
    for series_id in er_daily.columns:
        series = er_daily[series_id].copy().dropna().sort_index()
        annotations = find_local_min_max(series)
        _fig, ax = plt.subplots()
        annotate_chart_min_max(ax, annotations)
        title = er_daily_meta.loc[series_id, ra.rba_metacol.desc].replace(
            "; see notes for further detail.", ""
        )
        text = get_currency(title)
        ylabel = er_daily_meta.loc[series_id, ra.rba_metacol.unit] if not text else text
        line_plot(
            series,
            ax=ax,
            title=title,
            ylabel=ylabel,
            rfooter=f"RBA {table}",
            lfooter=f"Data to {series.index[-1]}. ",
            file_type=FILE_TYPE,
            pre_tag="f11.1-",
            width=1.5,
            dropna=False,
            show=SHOW,
        )


plot_exchange_rates()

In [26]:
def long_run_exchange_rates() -> None:
    """Plot long-run exchange rates."""

    # data capture and collation
    pd.set_option("future.no_silent_downcasting", True)
    catmap = ra.rba_catalogue()
    tables = catmap[catmap.index.str.contains("Z:F11.1-Daily")].index
    frame = pd.DataFrame()
    names = {}
    units = {}
    replacements = {"Closed": np.nan, "": np.nan, " --": np.nan, "CLOSED": np.nan}
    for table in tables:
        data, meta = ra.read_rba_table(table)
        data = data.loc[:, data.columns.notna()]  # drop nan named columns
        data = data.replace(replacements)
        data = data.dropna(how="all", axis=0)  # drop all nan rows
        data = data.astype(float)
        for index in meta.index:
            if index not in names:
                names[index] = meta.loc[index, ra.rba_metacol.desc].replace(
                    "; see notes for further detail.", ""
                )
            if index not in units:
                units[index] = meta.loc[index, ra.rba_metacol.unit]
        frame = pd.concat([frame, data], axis=0)
    frame = frame.sort_index()

    # plot

    for series_id in frame.columns:
        title = names[series_id]
        text = get_currency(title)
        ylabel = units[series_id] if not text else text
        line_plot(
            frame[series_id],
            title=title,
            ylabel=ylabel,
            rfooter="RBA",
            tags="-long-term",
            file_type=FILE_TYPE,
            pre_tag="f11.1-",
            dropna=False,
            show=SHOW,
        )


long_run_exchange_rates()

## Watermark

In [27]:
%load_ext watermark
%watermark --python --machine --conda --iversions --watermark

Python implementation: CPython
Python version       : 3.12.8
IPython version      : 8.31.0

conda environment: 312

Compiler    : Clang 18.1.8 
OS          : Darwin
Release     : 24.2.0
Machine     : arm64
Processor   : arm
CPU cores   : 14
Architecture: 64bit

matplotlib: 3.10.0
readabs   : 0.0.24
babel     : 2.16.0
pandas    : 2.2.3
numpy     : 1.26.4
seaborn   : 0.13.2

Watermark: 2.5.0



In [28]:
print("Done")

Done
